In [ ]:
from sagemaker.chainer import Chainer

In [ ]:
chainer_estimator = Chainer("train.py",
                        train_instance_type="ml.p3.2xlarge",
                        train_instance_count=1,
                        source_dir="custom_chainer",
                        hyperparameters = {'epoch': 20, 'batchsize': 64})
chainer_estimator.fit({'train': 's3://my-data-bucket/path/to/my/training/data',
                   'test': 's3://my-data-bucket/path/to/my/test/data'})